# Manipulating Data in DataFrames HW


#### Let's get started applying what we learned in the lecure!

I've provided several questions below to help test and expand you knowledge from the code along lecture. So let's see what you've got!

First create your spark instance as we need to do at the start of every project.

In [4]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('HW').getOrCreate()

In [5]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

## Read in our Republican vs. Democrats Tweet DataFrame

Attached to the lecture

In [6]:
df = spark.read.csv('Datasets/Rep_vs_Dem_tweets.csv',inferSchema=True,header=True)
df.limit(5).toPandas()

,Party,Handle,Tweet
0,Democrat,RepDarrenSoto,"Today, Senate Dems vote to #SaveTheInternet. P..."
1,Democrat,RepDarrenSoto,RT @WinterHavenSun: Winter Haven resident / Al...
2,Democrat,RepDarrenSoto,RT @NBCLatino: .@RepDarrenSoto noted that Hurr...
3,"Congress has allocated about $18…""",None,None
4,Democrat,RepDarrenSoto,RT @NALCABPolicy: Meeting with @RepDarrenSoto ...


## About this dataframe

Extracted tweets from all of the representatives (latest 200 as of May 17th 2018)

**Source:** https://www.kaggle.com/kapastor/democratvsrepublicantweets#ExtractedTweets.csv

Use either .show() or .toPandas() check out the first view rows of the dataframe to get an idea of what we are working with.

In [7]:
df.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



**Prevent Truncation of view**

If the view you produced above truncated some of the longer tweets, see if you can prevent that so you can read the whole tweet.

In [8]:
df.show(5,truncate=False)

+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Party                             |Handle       |Tweet                                                                                                                                       |
+----------------------------------+-------------+--------------------------------------------------------------------------------------------------------------------------------------------+
|Democrat                          |RepDarrenSoto|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L |
|Democrat                          |RepDarrenSoto|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|Democrat                          |RepD

**Print Schema**

First, check the schema to make sure the datatypes are accurate. 

In [9]:
df.printSchema()

root
 |-- Party: string (nullable = true)
 |-- Handle: string (nullable = true)
 |-- Tweet: string (nullable = true)



## 1. Can you identify any tweet that mentions the handle @LatinoLeader using regexp_extract?

It doesn't matter how you identify the row, any identifier will do. You can test your script on row 5 from this dataset. That row contains @LatinoLeader. 

In [10]:
def search_latino(x):
    try:
        if len(x.split('@LatinoLeader')) > 1:
            return True
        else:
            return False
    except:
        return False

search_udf = udf(lambda x: search_latino(x), BooleanType())
lat = df.withColumn('latino',search_udf('Tweet'))
lat.filter('latino == True').show(5)

+--------+-------------+--------------------+------+
|   Party|       Handle|               Tweet|latino|
+--------+-------------+--------------------+------+
|Democrat|RepDarrenSoto|RT @NALCABPolicy:...|  true|
+--------+-------------+--------------------+------+



In [11]:
# Other way to to it
df.withColumn('Latino',regexp_extract(df.Tweet, '(.)(@LatinoLeader)(.)',2)).show(5)

+--------------------+-------------+--------------------+-------------+
|               Party|       Handle|               Tweet|       Latino|
+--------------------+-------------+--------------------+-------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|             |
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|             |
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|             |
|Congress has allo...|         null|                null|         null|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|@LatinoLeader|
+--------------------+-------------+--------------------+-------------+
only showing top 5 rows



## 2. Replace any value other than 'Democrate' or 'Republican' with 'Other' in the Party column.

We can see from the output below, that there are several other values other than 'Democrate' or 'Republican' in the Part column. We are assuming that this is dirty data that needs to be cleaned up.

In [12]:
df.groupBy('Party').count().orderBy(desc('count')).show(5)

+--------------------+-----+
|               Party|count|
+--------------------+-----+
|          Republican|44392|
|            Democrat|42068|
|            That’s…"|   28|
|https://t.co/oc6J...|   22|
|                 Now|   17|
+--------------------+-----+
only showing top 5 rows



In [13]:
df.withColumn('correct_name', when(~df.Party.isin(['Democrat','Republican']),'other').otherwise(df.Party)).show(5)

+--------------------+-------------+--------------------+------------+
|               Party|       Handle|               Tweet|correct_name|
+--------------------+-------------+--------------------+------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|    Democrat|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|    Democrat|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|    Democrat|
|Congress has allo...|         null|                null|       other|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|    Democrat|
+--------------------+-------------+--------------------+------------+
only showing top 5 rows



## 3. Delete all embedded links (ie. "https:....)

For example see the first row in the tweets dataframe. 

*Note: this may require an google search :)*

In [14]:
test = df.select('Tweet').limit(1).collect()[0]['Tweet']
test

'Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House… https://t.co/n3tggDLU1L'

In [15]:
import string

def remove_url(x):
    try:
        return ' '.join([i for i in x.split(' ') if not i.startswith('https')])
    except:
        return 'no content'

remove_url(test)

'Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…'

In [16]:
clean_url = udf(lambda x: remove_url(x), StringType())
df.withColumn('cleaned_text', clean_url('Tweet')).select('cleaned_text').show(5,truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+
|cleaned_text                                                                                                                                |
+--------------------------------------------------------------------------------------------------------------------------------------------+
|Today, Senate Dems vote to #SaveTheInternet. Proud to support similar #NetNeutrality legislation here in the House…                         |
|RT @WinterHavenSun: Winter Haven resident / Alta Vista teacher is one of several recognized by @RepDarrenSoto for National Teacher Apprecia…|
|RT @NBCLatino: .@RepDarrenSoto noted that Hurricane Maria has left approximately $90 billion in damages.                                    |
|no content                                                                                                                                  |

## 4. Remove any leading or trailing white space in the tweet column

In [17]:
df.withColumn('trimmed_tweet', trim('Tweet')).show(5)

+--------------------+-------------+--------------------+--------------------+
|               Party|       Handle|               Tweet|       trimmed_tweet|
+--------------------+-------------+--------------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|RT @NBCLatino: .@...|
|Congress has allo...|         null|                null|                null|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|RT @NALCABPolicy:...|
+--------------------+-------------+--------------------+--------------------+
only showing top 5 rows



## 5. Rename the 'Party' column to 'Dem_Rep'

No real reason here :) just wanted you to get practice doing this. 

In [18]:
df.withColumnRenamed('Party','Dem_Rep').show(5)

+--------------------+-------------+--------------------+
|             Dem_Rep|       Handle|               Tweet|
+--------------------+-------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|
|Congress has allo...|         null|                null|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|
+--------------------+-------------+--------------------+
only showing top 5 rows



## 6. Concatenate the Party and Handle columns

Silly yes... but good practice.

pyspark.sql.functions.concat_ws(sep, *cols)[source] <br>
Concatenates multiple input string columns together into a single string column, using the given separator.

In [19]:
df.withColumn('concat', concat_ws(' - ','Party','Handle')).show(5)

+--------------------+-------------+--------------------+--------------------+
|               Party|       Handle|               Tweet|              concat|
+--------------------+-------------+--------------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|Democrat - RepDar...|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|Democrat - RepDar...|
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|Democrat - RepDar...|
|Congress has allo...|         null|                null|Congress has allo...|
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...|Democrat - RepDar...|
+--------------------+-------------+--------------------+--------------------+
only showing top 5 rows



## Challenge Question

Let's image that we want to analyze the hashtags that are used in these tweets. Can you extract all the hashtags you see?

In [20]:
def extract_hashtags(x):
    try:
        return ' '.join([i for i in x.split(' ') if i.startswith('#')])
    except:
        return ''

extract_hashtags(test)
hastags_udf = udf(lambda x: extract_hashtags(x), StringType())

df.withColumn('Hastags',hastags_udf('Tweet')).show(10)

+--------------------+-------------+--------------------+--------------------+
|               Party|       Handle|               Tweet|             Hastags|
+--------------------+-------------+--------------------+--------------------+
|            Democrat|RepDarrenSoto|Today, Senate Dem...|#SaveTheInternet....|
|            Democrat|RepDarrenSoto|RT @WinterHavenSu...|                    |
|            Democrat|RepDarrenSoto|RT @NBCLatino: .@...|                    |
|Congress has allo...|         null|                null|                    |
|            Democrat|RepDarrenSoto|RT @NALCABPolicy:...| #NALCABPolicy2018.…|
|            Democrat|RepDarrenSoto|RT @Vegalteno: Hu...|                    |
|            Democrat|RepDarrenSoto|RT @EmgageActionF...|                    |
|            Democrat|RepDarrenSoto|Hurricane Maria l...|                    |
|            Democrat|RepDarrenSoto|RT @Tharryry: I a...|      #NetNeutrality|
|            Democrat|RepDarrenSoto|RT @HispanicCauc

# Let's create our own dataset to work with real dates

This is a dataset of patient visits from a medical office. It contains the patients first and last names, date of birth, and the dates of their first 3 visits. 

In [21]:
from pyspark.sql.types import *

md_office = [('Mohammed','Alfasy','1987-4-8','2016-1-7','2017-2-3','2018-3-2') \
            ,('Marcy','Wellmaker','1986-4-8','2015-1-7','2017-1-3','2018-1-2') \
            ,('Ginny','Ginger','1986-7-10','2014-8-7','2015-2-3','2016-3-2') \
            ,('Vijay','Doberson','1988-5-2','2016-1-7','2018-2-3','2018-3-2') \
            ,('Orhan','Gelicek','1987-5-11','2016-5-7','2017-1-3','2018-9-2') \
            ,('Sarah','Jones','1956-7-6','2016-4-7','2017-8-3','2018-10-2') \
            ,('John','Johnson','2017-10-12','2018-1-2','2018-10-3','2018-3-2') ]

df = spark.createDataFrame(md_office,['first_name','last_name','dob','visit1','visit2','visit3']) # schema=final_struc

# Check to make sure it worked
df.show()
print(df.printSchema())

+----------+---------+----------+--------+---------+---------+
|first_name|last_name|       dob|  visit1|   visit2|   visit3|
+----------+---------+----------+--------+---------+---------+
|  Mohammed|   Alfasy|  1987-4-8|2016-1-7| 2017-2-3| 2018-3-2|
|     Marcy|Wellmaker|  1986-4-8|2015-1-7| 2017-1-3| 2018-1-2|
|     Ginny|   Ginger| 1986-7-10|2014-8-7| 2015-2-3| 2016-3-2|
|     Vijay| Doberson|  1988-5-2|2016-1-7| 2018-2-3| 2018-3-2|
|     Orhan|  Gelicek| 1987-5-11|2016-5-7| 2017-1-3| 2018-9-2|
|     Sarah|    Jones|  1956-7-6|2016-4-7| 2017-8-3|2018-10-2|
|      John|  Johnson|2017-10-12|2018-1-2|2018-10-3| 2018-3-2|
+----------+---------+----------+--------+---------+---------+

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: string (nullable = true)
 |-- visit1: string (nullable = true)
 |-- visit2: string (nullable = true)
 |-- visit3: string (nullable = true)

None


Oh no! The dates are still stored as text... let's try converting them again and see if we have any issues this time.

In [22]:
df = df.withColumn('dob', df.dob.cast(DateType())) \
    .withColumn('visit1', df.visit1.cast(DateType())) \
    .withColumn('visit2', df.visit2.cast(DateType())) \
    .withColumn('visit3', df.visit3.cast(DateType()))

df.printSchema()

root
 |-- first_name: string (nullable = true)
 |-- last_name: string (nullable = true)
 |-- dob: date (nullable = true)
 |-- visit1: date (nullable = true)
 |-- visit2: date (nullable = true)
 |-- visit3: date (nullable = true)



## 7. Can you calculate a variable showing the length of time between patient visits?

Compare visit1 to visit2 and visit2 to visit3 for all patients and see what the average length of time is between visits. Create an alias for it as well. 

In [23]:
df.withColumn('v1-v2',datediff(df.visit2,df.visit1)).withColumn('v2-v3',datediff(df.visit3,df.visit2)).show(5)

+----------+---------+----------+----------+----------+----------+-----+-----+
|first_name|last_name|       dob|    visit1|    visit2|    visit3|v1-v2|v2-v3|
+----------+---------+----------+----------+----------+----------+-----+-----+
|  Mohammed|   Alfasy|1987-04-08|2016-01-07|2017-02-03|2018-03-02|  393|  392|
|     Marcy|Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02|  727|  364|
|     Ginny|   Ginger|1986-07-10|2014-08-07|2015-02-03|2016-03-02|  180|  393|
|     Vijay| Doberson|1988-05-02|2016-01-07|2018-02-03|2018-03-02|  758|   27|
|     Orhan|  Gelicek|1987-05-11|2016-05-07|2017-01-03|2018-09-02|  241|  607|
+----------+---------+----------+----------+----------+----------+-----+-----+
only showing top 5 rows



## 8. Can you calculate the age of each patient?

In [25]:
df.withColumn('age',ceil(datediff(current_date(),df.dob)/365)).show(5) # ceil = round

+----------+---------+----------+----------+----------+----------+---+
|first_name|last_name|       dob|    visit1|    visit2|    visit3|age|
+----------+---------+----------+----------+----------+----------+---+
|  Mohammed|   Alfasy|1987-04-08|2016-01-07|2017-02-03|2018-03-02| 34|
|     Marcy|Wellmaker|1986-04-08|2015-01-07|2017-01-03|2018-01-02| 35|
|     Ginny|   Ginger|1986-07-10|2014-08-07|2015-02-03|2016-03-02| 35|
|     Vijay| Doberson|1988-05-02|2016-01-07|2018-02-03|2018-03-02| 33|
|     Orhan|  Gelicek|1987-05-11|2016-05-07|2017-01-03|2018-09-02| 34|
+----------+---------+----------+----------+----------+----------+---+
only showing top 5 rows



## 9. Can you extract the month from the first visit column and call it "Month"?

In [26]:
df.select(month('visit1').alias('month')).show(5)

+-----+
|month|
+-----+
|    1|
|    1|
|    8|
|    1|
|    5|
+-----+
only showing top 5 rows



## 10. Challenges with working with date and timestamps

Let's read in the supermarket sales dataframe attached to the lecture now and see some of the issues that can come up when working with date and timestamps values.

In [42]:
spm = spark.read.csv('Datasets/supermarket_sales.csv', inferSchema=True,header=True)
spm.limit(5).toPandas()

,Invoice ID,Branch,City,Customer type,Gender,Product line,Unit price,Quantity,Tax 5%,Total,Date,Time,Payment,cogs,gross margin percentage,gross income,Rating
0,750-67-8428,A,Yangon,Member,Female,Health and beauty,74.69,7,26.1415,548.9715,1/5/2019,13:08,Ewallet,522.83,4.761905,26.1415,9.1
1,226-31-3081,C,Naypyitaw,Normal,Female,Electronic accessories,15.28,5,3.8200,80.2200,3/8/2019,10:29,Cash,76.40,4.761905,3.8200,9.6
2,631-41-3108,A,Yangon,Normal,Male,Home and lifestyle,46.33,7,16.2155,340.5255,3/3/2019,13:23,Credit card,324.31,4.761905,16.2155,7.4
3,123-19-1176,A,Yangon,Member,Male,Health and beauty,58.22,8,23.2880,489.0480,1/27/2019,20:33,Ewallet,465.76,4.761905,23.2880,8.4
4,373-73-7910,A,Yangon,Normal,Male,Sports and travel,86.31,7,30.2085,634.3785,2/8/2019,10:37,Ewallet,604.17,4.761905,30.2085,5.3


## About this dataset

The growth of supermarkets in most populated cities are increasing and market competitions are also high. The dataset is one of the historical sales of supermarket company which has recorded in 3 different branches for 3 months data. 

 - Attribute information
 - Invoice id: Computer generated sales slip invoice identification number
 - Branch: Branch of supercenter (3 branches are available identified by A, B and C).
 - City: Location of supercenters
 - Customer type: Type of customers, recorded by Members for customers using member card and Normal for without member card.
 - Gender: Gender type of customer
 - Product line: General item categorization groups - Electronic accessories, Fashion accessories, Food and beverages, Health and beauty, Home and lifestyle, Sports and travel
 - Unit price: Price of each product in USD
 - Quantity: Number of products purchased by customer
 - Tax: 5% tax fee for customer buying
 - Total: Total price including tax
 - Date: Date of purchase (Record available from January 2019 to March 2019)
 - Time: Purchase time (10am to 9pm)
 - Payment: Payment used by customer for purchase (3 methods are available – Cash, Credit card and Ewallet)
 - COGS: Cost of goods sold
 - Gross margin percentage: Gross margin percentage
 - Gross income: Gross income
 - Rating: Customer stratification rating on their overall shopping experience (On a scale of 1 to 10)

**Source:** https://www.kaggle.com/aungpyaeap/supermarket-sales

### View dataframe and schema as usual

In [43]:
spm.printSchema()

root
 |-- Invoice ID: string (nullable = true)
 |-- Branch: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Customer type: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Product line: string (nullable = true)
 |-- Unit price: double (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- Tax 5%: double (nullable = true)
 |-- Total: double (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Payment: string (nullable = true)
 |-- cogs: double (nullable = true)
 |-- gross margin percentage: double (nullable = true)
 |-- gross income: double (nullable = true)
 |-- Rating: double (nullable = true)



### Convert date field to date type

Looks like we need to convert the date field into a date type. Let's go ahead and do that..

In [49]:
spm.withColumn('Date', to_date('Date')).select('Date').show(5)

+----+
|Date|
+----+
|null|
|null|
|null|
|null|
|null|
+----+
only showing top 5 rows



In [56]:
def correct_date(x):
    try:
        return x.split('/')[2]+'-'+x.split('/')[0]+'-'+x.split('/')[1]
    except:
        return ''

udf_dates = udf(lambda x: correct_date(x), StringType())
spm = spm.withColumn('Date', to_date(udf_dates('Date')))
spm.select('Date').show(5)

+----------+
|      Date|
+----------+
|2019-01-05|
|2019-03-08|
|2019-03-03|
|2019-01-27|
|2019-02-08|
+----------+
only showing top 5 rows



### How can we extract the month value from the date field?

If you had trouble converting the date field in the previous question think about a more creative solution to extract the month from that field.

In [57]:
spm.select(month('Date').alias('month')).show(5)

+-----+
|month|
+-----+
|    1|
|    3|
|    3|
|    1|
|    2|
+-----+
only showing top 5 rows



## 11.0 Working with Arrays

Here is a dataframe of reviews from the movie the Dark Night.

In [58]:
from pyspark.sql.functions import *

values = [(5,'Epic. This is the best movie I have EVER seen'), \
          (4,'Pretty good, but I would have liked to seen better special effects'), \
          (3,'So so. Casting could have been improved'), \
          (5,'The most EPIC movie of the year! Casting was awesome. Special effects were so intense.'), \
          (4,'Solid but I would have liked to see more of the love story'), \
          (5,'THE BOMB!!!!!!!')]
reviews = spark.createDataFrame(values,['rating', 'review_txt'])

reviews.show(6,False)

+------+--------------------------------------------------------------------------------------+
|rating|review_txt                                                                            |
+------+--------------------------------------------------------------------------------------+
|5     |Epic. This is the best movie I have EVER seen                                         |
|4     |Pretty good, but I would have liked to seen better special effects                    |
|3     |So so. Casting could have been improved                                               |
|5     |The most EPIC movie of the year! Casting was awesome. Special effects were so intense.|
|4     |Solid but I would have liked to see more of the love story                            |
|5     |THE BOMB!!!!!!!                                                                       |
+------+--------------------------------------------------------------------------------------+



## 11.1 Let's see if we can create an array off of the review text column and then derive some meaningful results from it.

**But first** we need to clean the rview_txt column to make sure we can get what we need from our analysis later on. So let's do the following:

1. Remove all punctuation
2. lower case everything
3. Remove white space (trim)
3. Then finally, split the string

In [66]:
reviews = reviews.withColumn('cleaned_text',regexp_replace('review_txt','[^\sa-zA-Z0-9]',''))
reviews = reviews.withColumn('cleaned_text',trim(lower('cleaned_text')))
reviews = reviews.withColumn('cleaned_text',split('cleaned_text',' '))
reviews.show(5)

+------+--------------------+--------------------+
|rating|          review_txt|        cleaned_text|
+------+--------------------+--------------------+
|     5|Epic. This is the...|[epic, this, is, ...|
|     4|Pretty good, but ...|[pretty, good, bu...|
|     3|So so. Casting co...|[so, so, casting,...|
|     5|The most EPIC mov...|[the, most, epic,...|
|     4|Solid but I would...|[solid, but, i, w...|
+------+--------------------+--------------------+
only showing top 5 rows



## 11.2 Alright now let's see if we can find which reviews contain the word 'Epic'

In [67]:
reviews.withColumn('Res',array_contains('cleaned_text','epic')).filter('Res == True').show()

+------+--------------------+--------------------+----+
|rating|          review_txt|        cleaned_text| Res|
+------+--------------------+--------------------+----+
|     5|Epic. This is the...|[epic, this, is, ...|true|
|     5|The most EPIC mov...|[the, most, epic,...|true|
+------+--------------------+--------------------+----+



### That's it! Great Job!